In [7]:
import numpy as np

# Activation Functions


In [8]:
class ReLU:
    def forward(self, x):
        self.input = x
        self.output = np.maximum(0, x)
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.input <= 0] = 0
        return self.dinputs


class LReLU:
    def __init__(self, alpha=0.01):
        self.alpha = alpha

    def forward(self, x):
        self.input = x
        self.output = np.where(x > 0, x, self.alpha * x)
        return self.output

    def backward(self, dvalues):
        self.dinputs = np.where(self.input > 0, dvalues, dvalues * self.alpha)
        return self.dinputs


class Sigmoid:
    def forward(self, x):
        self.input = x
        self.output = 1 / (1 + np.exp(-x))
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))
        return self.dinputs


class Tanh:
    def forward(self, x):
        self.input = x
        self.output = np.tanh(x)
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)
        return self.dinputs


class Softmax:
    def forward(self, x):
        exp_values = np.exp(x - np.max(x, axis=1, keepdims=True)) 
        self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        return self.dinputs


class Linear:
    def forward(self, x):
        self.input = x
        self.output = x
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        return self.dinputs



In [9]:
x=np.array([[1, -2, 3], [-1, 2, -3]])
relu = ReLU()
lrelu=LReLU()
linear=Linear()
sigmoid=Sigmoid()
tanh=Tanh()
softmax=Softmax()

In [10]:
print("Relu :",relu.forward(x))
print("LReLU :",lrelu.forward(x))
print("Linear :",linear.forward(x))
print("Sigmoid :",sigmoid.forward(x))
print("Tanh :",tanh.forward(x))
print("Softmax :",softmax.forward(x))

Relu : [[1 0 3]
 [0 2 0]]
LReLU : [[ 1.   -0.02  3.  ]
 [-0.01  2.   -0.03]]
Linear : [[ 1 -2  3]
 [-1  2 -3]]
Sigmoid : [[0.73105858 0.11920292 0.95257413]
 [0.26894142 0.88079708 0.04742587]]
Tanh : [[ 0.76159416 -0.96402758  0.99505475]
 [-0.76159416  0.96402758 -0.99505475]]
Softmax : [[0.11849965 0.00589975 0.8756006 ]
 [0.04712342 0.94649912 0.00637746]]


# Layers

In [13]:
import numpy as np

class InputLayer:

    def forward(self, X):
        self.output = X
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues
        return self.dinputs


class Dense:

    def __init__(self, input_size, output_size, activation=None, learning_rate=0.01, weight_init="random"):
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = learning_rate


        if weight_init == "random":
            self.weights = np.random.randn(input_size, output_size) * 0.01
        elif weight_init == "xavier":
            self.weights = np.random.randn(input_size, output_size) * np.sqrt(2 / (input_size + output_size))
        else:
            raise ValueError("Unknown weight initialization type")


        self.biases = np.zeros((1, output_size))

    
        self.activation = activation

    def forward(self, X):
        
        self.input = X

        self.linear_output = np.dot(X, self.weights) + self.biases
class Dropout:
    def __init__(self, rate):
        self.rate = 1 - rate
    def forward(self, inputs):
        self.mask = (np.random.rand(*inputs.shape) < self.rate) / self.rate
        self.output = inputs * self.mask
    def backward(self, dvalues):
        self.dinputs = dvalues * self.mask


        if self.activation is not None:
            self.output = self.activation.forward(self.linear_output)
        else:
            self.output = self.linear_output

        return self.output

    def backward(self, dvalues):

        if self.activation is not None:
            dactivation = self.activation.backward(dvalues)
        else:
            dactivation = dvalues

        self.dweights = np.dot(self.input.T, dactivation)     
        self.dbiases = np.sum(dactivation, axis=0, keepdims=True)  
        self.dinputs = np.dot(dactivation, self.weights.T)      

        self.weights -= self.learning_rate * self.dweights
        self.biases  -= self.learning_rate * self.dbiases

        return self.dinputs
 
 

In [14]:
from sklearn.metrics import mean_squared_error

In [ ]:
np.random.seed(42)
X = np.random.randn(100, 2)
Y = (X[:, 0] * 0.5 + X[:, 1] * -0.2 + 0.1 > 0).astype(int).reshape(-1, 1)

# Define layers
dense1 = Dense(2, 64)
activation1 = ReLU()
dropout1 = Dropout(rate=0.2)

dense2 = Dense(64, 1)
activation2 = Sigmoid()

loss_function = mean_squared_error()

# Training loop
epochs = 500
learning_rate = 0.01

for epoch in range(epochs):
    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    dropout1.forward(activation1.output)

    dense2.forward(dropout1.output)
    activation2.forward(dense2.output)

    loss = loss_function.forward(activation2.output, Y)

    # Backward pass
    loss_function.backward(activation2.output, Y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)
    dropout1.backward(dense2.dinputs)
    activation1.backward(dropout1.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights
    dense1.weights -= learning_rate * dense1.dweights
    dense1.biases  -= learning_rate * dense1.dbiases
    dense2.weights -= learning_rate * dense2.dweights
    dense2.biases  -= learning_rate * dense2.dbiases

    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


array([[ 9.15200428e-06,  3.11552640e-05,  3.60918932e-06],
       [ 1.32289770e-03,  4.50340994e-03,  5.21698647e-04],
       [-2.92781942e-04, -9.96688638e-04, -1.15461644e-04],
       [ 1.77450817e-04,  6.04078285e-04,  6.99795995e-05],
       [ 1.49402123e-04,  5.08594885e-04,  5.89183013e-05]])